In [1]:
from sympy import Symbol, Function, Matrix, zeros, symbols
import sympy
import numpy as np
import matplotlib.pyplot as plt

Q matrix:

$$
\begin{aligned}
Q[\mathbf{x}, \delta \mathbf{x}] & = \sum_{i,j}^d \sum_{k,l}^\infty d^i_k d_l^j \int_0^T dt \left[
\sqrt{\lambda_k \lambda_l} \phi_k \phi_l \frac{\partial^2 L}{\partial x_i \partial x_j}
+ \sqrt{\lambda_l} \psi_k \phi_l \frac{\partial^2 L}{\partial \dot{x}_i \partial x_j}
+ \psi_k \psi_l \frac{\partial^2 L}{\partial \dot{x}_i \partial \dot{x}_j}
\right] \\
& = \sum_{i,j}^d \sum_{k,l}^\infty d^i_k d_l^j Q^{ij}_{kl} \\
S[\mathbf{x}^* + \delta \mathbf{x}] & = S[\mathbf{x}^*] + \frac{1}{2} d^T Q d
\end{aligned}
$$

In [2]:
dim = 1

In [7]:
t = Symbol('t')
T = Symbol('T', nonnegative=True)
k = Symbol('k')
g = Symbol('gamma')
b = Symbol('beta')

x = Symbol('x')
xdot = Symbol('xdot')

coords = [x]
dcoords = [xdot]

U = k * x**2
F = -U.diff(x)
F = Function('F')(x)

L_OM = (b*g/4) * (xdot - F/g)**2 + (1/(2*g)) * F.diff(x)
L_FW = (b*g/4) * (xdot - F/g)**2

F_expr = -U.diff(x)
div_F_expr = F_expr.diff(x)

### Second variation

In [10]:
def get_L_2nd_derivs(L):
    L_x_x = zeros(dim,dim)
    L_xd_x = zeros(dim,dim)
    L_xd_xd = zeros(dim,dim)

    for i in range(dim):
        for j in range(dim):
            L_x_x[i,j] = L.diff(coords[i]).diff(coords[j])
            L_x_x[i,j] = L_x_x[i,j].subs(F, F_expr).simplify()

            L_xd_x[i,j] = L.diff(dcoords[i]).diff(coords[j])
            L_xd_x[i,j] = L_xd_x[i,j].subs(F, F_expr).simplify()

            L_xd_xd[i,j] = L.diff(dcoords[i]).diff(dcoords[j])
            L_xd_xd[i,j] = L_xd_xd[i,j].subs(F, F_expr).simplify()
            
    return L_x_x, L_xd_x, L_xd_xd

L_x_x_OM, L_xd_x_OM, L_xd_xd_OM = get_L_2nd_derivs(L_OM)
L_x_x_FW, L_xd_x_FW, L_xd_xd_FW = get_L_2nd_derivs(L_FW)

In [11]:
def to_pycode(s):
    return sympy.printing.pycode(s)

In [12]:
py_force_exprs = [U, F_expr, div_F_expr]
py_force_exprs = list(map(to_pycode, py_force_exprs))

py_exprs = []

for i in range(dim):
    for j in range(dim):
        py_exprs.append(L_x_x_OM[i,j])
        py_exprs.append(L_xd_x_OM[i,j])
        py_exprs.append(L_xd_xd_OM[i,j])
        
for i in range(dim):
    for j in range(dim):
        py_exprs.append(L_x_x_FW[i,j])
        py_exprs.append(L_xd_x_FW[i,j])
        py_exprs.append(L_xd_xd_FW[i,j])
        
py_exprs = list(map(to_pycode, py_exprs))

In [29]:
L_2nd_derivs_OM_str = """
def L_2nd_derivs_OM(x, xdot):
    L_x_x = np.zeros((%s, %s, len(x)))
    L_xd_x = np.zeros((%s, %s, len(x)))
    L_xd_xd = np.zeros((%s, %s, len(x)))
""" % (dim, dim, dim, dim, dim, dim)

for i in range(dim):
    for j in range(dim):
        L_2nd_derivs_OM_str += "    L_x_x[%s,%s] = %s\n" % (i,j, to_pycode(L_x_x_OM[i,j]))
        L_2nd_derivs_OM_str += "    L_xd_x[%s,%s] = %s\n" % (i,j, to_pycode(L_xd_x_OM[i,j]))
        L_2nd_derivs_OM_str += "    L_xd_xd[%s,%s] = %s\n" % (i,j, to_pycode(L_xd_xd_OM[i,j]))

L_2nd_derivs_OM_str += "    return L_x_x, L_xd_x, L_xd_xd"

In [30]:
L_2nd_derivs_FW_str = """
def L_2nd_derivs_FW(x, xdot):
    L_x_x = np.zeros((%s, %s, len(x)))
    L_xd_x = np.zeros((%s, %s, len(x)))
    L_xd_xd = np.zeros((%s, %s, len(x)))
""" % (dim, dim, dim, dim, dim, dim)

for i in range(dim):
    for j in range(dim):
        L_2nd_derivs_FW_str += "    L_x_x[%s,%s] = %s\n" % (i,j, to_pycode(L_x_x_FW[i,j]))
        L_2nd_derivs_FW_str += "    L_xd_x[%s,%s] = %s\n" % (i,j, to_pycode(L_xd_x_FW[i,j]))
        L_2nd_derivs_FW_str += "    L_xd_xd[%s,%s] = %s\n" % (i,j, to_pycode(L_xd_xd_FW[i,j]))
        
L_2nd_derivs_FW_str += "    return L_x_x, L_xd_x, L_xd_xd"

In [31]:
print("""
def Upot(x):
    return %s
def force_func(x):
    f = %s
    div_f = %s
    return f, div_f
def force_vec(x):
    f, div_f = force_func(x)
    return f
""" % tuple(py_force_exprs))


def Upot(x):
    return k*x**2
def force_func(x):
    f = -2*k*x
    div_f = -2*k
    return f, div_f
def force_vec(x):
    f, div_f = force_func(x)
    return f



In [32]:
print("%s%s" % (L_2nd_derivs_OM_str, L_2nd_derivs_FW_str))


def L_2nd_derivs_OM(x, xdot):
    L_x_x = np.zeros((1, 1, len(x)))
    L_xd_x = np.zeros((1, 1, len(x)))
    L_xd_xd = np.zeros((1, 1, len(x)))
    L_x_x[0,0] = 2*beta*k**2/gamma
    L_xd_x[0,0] = beta*k
    L_xd_xd[0,0] = (1/2)*beta*gamma
    return L_x_x, L_xd_x, L_xd_xd
def L_2nd_derivs_FW(x, xdot):
    L_x_x = np.zeros((1, 1, len(x)))
    L_xd_x = np.zeros((1, 1, len(x)))
    L_xd_xd = np.zeros((1, 1, len(x)))
    L_x_x[0,0] = 2*beta*k**2/gamma
    L_xd_x[0,0] = beta*k
    L_xd_xd[0,0] = (1/2)*beta*gamma
    return L_x_x, L_xd_x, L_xd_xd


### Gradient of the action

In [18]:
dim = 1

In [19]:
g = Symbol('gamma')
b = Symbol('beta')

x, y = symbols('x y')
xdot, ydot = symbols('xdot ydot')

coords = [x]
dcoords = [xdot]

In [20]:
U0 = Symbol('U0')
dU = Symbol('dU')

x = Symbol('x')
xdot = Symbol('xdot')

U = ( U0 * (x-1)**2 - dU * (x-2) / 4 ) * (x+1)**2
F = -U.diff(x)
divF = F.diff(x)

In [21]:
L_OM = (b*g/4) * (xdot - F/g)**2 + (1/(2*g)) * divF
L_FW = (b*g/4) * (xdot - F/g)**2

L_OM = L_OM.simplify()
L_FW = L_FW.simplify()

In [22]:
def get_L_derivs(L):
    L_x = zeros(dim, 1)
    L_xd = zeros(dim, 1)

    for i in range(dim):
        L_x[i] = L.diff(coords[i]).simplify()
        
        L_xd[i] = L.diff(dcoords[i]).simplify()
            
    return L_x, L_xd

L_x_OM, L_xd_OM = get_L_derivs(L_OM)
L_x_FW, L_xd_FW = get_L_derivs(L_FW)

In [23]:
def to_pycode(s):
    return sympy.printing.pycode(s)

In [24]:
#py_force_exprs = [U_expr, F_expr, divF_expr]
py_force_exprs = list(map(to_pycode, py_force_exprs))

py_exprs = []

for i in range(dim):
    py_exprs.append(L_x_OM[i])
    py_exprs.append(L_xd_OM[i])
        
for i in range(dim):
    py_exprs.append(L_x_FW[i])
    py_exprs.append(L_xd_FW[i])
        
py_exprs = list(map(to_pycode, py_exprs))

In [25]:
L_derivs_OM_str = """
def L_derivs_OM(x, xdot):
    L_x = np.zeros((%s, len(x)))
    L_xd = np.zeros((%s, len(x)))
""" % (dim, dim)

for i in range(dim):
    L_derivs_OM_str += "    L_x[%s] = %s\n" % (i, to_pycode(L_x_OM[i]))
    L_derivs_OM_str += "    L_xd[%s] = %s\n" % (i, to_pycode(L_xd_OM[i]))

L_derivs_OM_str += "    return L_x, L_xd"

In [26]:
L_derivs_FW_str = """
def L_derivs_FW(x, xdot):
    L_x = np.zeros((%s, len(x)))
    L_xd = np.zeros((%s, len(x)))
""" % (dim, dim)

for i in range(dim):
    L_derivs_FW_str += "    L_x[%s] = %s\n" % (i, to_pycode(L_x_FW[i]))
    L_derivs_FW_str += "    L_xd[%s] = %s\n" % (i, to_pycode(L_xd_FW[i]))

L_derivs_FW_str += "    return L_x, L_xd"

In [27]:
print("%s%s" % (L_derivs_OM_str, L_derivs_FW_str))


def L_derivs_OM(x, xdot):
    L_x = np.zeros((1, len(x)))
    L_xd = np.zeros((1, len(x)))
    L_x[0] = (1/16)*(-96*U0*(x - 1) - 96*U0*(x + 1) + beta*(4*gamma*xdot + (x + 1)**2*(8*U0*(x - 1) - dU) + 2*(x + 1)*(4*U0*(x - 1)**2 - dU*(x - 2)))*(4*U0*(x - 1)**2 + 4*U0*(x + 1)**2 - dU*(x - 2) + 2*(x + 1)*(8*U0*(x - 1) - dU)) + 12*dU)/gamma
    L_xd[0] = (1/8)*beta*(16*U0*x**3 - 16*U0*x - 3*dU*x**2 + 3*dU + 4*gamma*xdot)
    return L_x, L_xd
def L_derivs_FW(x, xdot):
    L_x = np.zeros((1, len(x)))
    L_xd = np.zeros((1, len(x)))
    L_x[0] = (1/16)*beta*(4*gamma*xdot + (x + 1)**2*(8*U0*(x - 1) - dU) + 2*(x + 1)*(4*U0*(x - 1)**2 - dU*(x - 2)))*(4*U0*(x - 1)**2 + 4*U0*(x + 1)**2 - dU*(x - 2) + 2*(x + 1)*(8*U0*(x - 1) - dU))/gamma
    L_xd[0] = (1/8)*beta*(16*U0*x**3 - 16*U0*x - 3*dU*x**2 + 3*dU + 4*gamma*xdot)
    return L_x, L_xd
